# Apache Zookeeper

- http://kazoo.readthedocs.io/

In [1]:
%matplotlib inline

In [2]:
import os
import pandas as pd
from kazoo.client import KazooClient

In [3]:
zk = KazooClient(hosts="node-master:2181")
zk.start()
zk.state

'CONNECTED'

In [4]:
zk.server_version()

(3, 4, 10)

In [5]:
def get_paths(zk, path, level=0):
    path_list = []
    for child in zk.get_children(path):
        child_path = os.path.join(path, child)
        print(child_path)
        
        value, znodestat = None, None
        try:
            value, znodestat = zk.get(child_path)
        except Exception as e:
            print(type(e), e)
        
        path_list.append(dict(
            path=child_path, 
            value=value, 
            znodestat=znodestat
        ))
        if level < 2:
            path_list.extend(get_paths(zk, child_path, level=level+1))
    
    return path_list

In [9]:
paths = get_paths(zk, '/hbase-unsecure')

/hbase-unsecure/meta-region-server
/hbase-unsecure/rs
/hbase-unsecure/rs/node-master,16020,1600419105559
/hbase-unsecure/splitWAL
/hbase-unsecure/backup-masters
/hbase-unsecure/flush-table-proc
/hbase-unsecure/flush-table-proc/abort
/hbase-unsecure/flush-table-proc/acquired
/hbase-unsecure/flush-table-proc/reached
/hbase-unsecure/master-maintenance
/hbase-unsecure/online-snapshot
/hbase-unsecure/online-snapshot/abort
/hbase-unsecure/online-snapshot/acquired
/hbase-unsecure/online-snapshot/reached
/hbase-unsecure/master
/hbase-unsecure/switch
/hbase-unsecure/running
/hbase-unsecure/draining
/hbase-unsecure/namespace
/hbase-unsecure/namespace/default
/hbase-unsecure/namespace/hbase
/hbase-unsecure/hbaseid
/hbase-unsecure/table
/hbase-unsecure/table/gdelt_custom_gdelt_5fcustom_z3_geom_event_5fdate_v6
/hbase-unsecure/table/hbase:meta
/hbase-unsecure/table/gdelt_custom_gdelt_5fcustom_id_v4
/hbase-unsecure/table/hbase:namespace
/hbase-unsecure/table/gdelt_custom_gdelt_5fcustom_z2_geom_v5
/hb

In [10]:
zk.stop()
zk.close()